In [18]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.cpu, "/O2 /fp:fast /openmp")

def modified_gram_schmidt(A):
    """
    Implements the Modified Gram-Schmidt orthogonalization to get the QR decomposition of matrix A.
    A = QR
    """
    A = A.astype(float)  # Ensure A is of float type
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for i in range(n-1):
        R[i, i] = np.linalg.norm(A[:, i])
        Q[:, i] = A[:, i] / R[i, i]
        R[i, i+1:n] = np.dot(Q[:, i].T, A[:, i+1:n])
        A[:, i+1:n] -= np.outer(Q[:, i], R[i, i+1:n])
    R[n-1, n-1] = np.linalg.norm(A[:, n-1])
    Q[:, n-1] = A[:, n-1] / R[n-1, n-1]
    return Q, R

# Householder reflection
def householder_reflection(a):
    v = a.copy()
    v[0] = v[0] + np.copysign(np.linalg.norm(v), a[0])
    v = v / np.linalg.norm(v)
    return np.eye(len(a)) - 2 * np.outer(v, v)

# QR decomposition using Householder reflections
def qr_decomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for i in range(min(m, n)):
        H = np.eye(m)
        H[i:, i:] = householder_reflection(R[i:, i])
        Q = Q @ H
        R = H @ R
    return Q, R

#dynamic size QR decomposition
def QRDecomposition():
    A = tf.input([-1, -1], tf.float32)

    m, n = A.shape
    Q = tf.zeros([m, n])
    R = tf.zeros([n, n])

    j = tf.index(0, [m])

    def loop_body(i):
        norm2 = tf.zeros([1], tf.float32)
        def loop_body1(it):
            norm2.set(norm2 + A[it, i] ** 2)
        tf.loop(loop_body1, 0, m, 1)
        R[i, i] = tf.sqrt(norm2)
        Q[j, i] = A[j, i] / R[i, i]
        
        t, = tf.index_grid([i+1], [n])
        dotprod = tf.zeros(t.shape, tf.float32)
        def loop_body2(it):
            dotprod.set(dotprod + Q[it, i] * A[it, t])
        tf.loop(loop_body2, 0, m, 1)
        R[i, t] = dotprod
        
        p, k = tf.index_grid([0, i+1], [m, n])
        A[p, k] -= Q[p, i] * R[i, k]

    tf.loop(loop_body, 0, n-1, 1)

    norm2 = tf.zeros([1], tf.float32)
    def loop_body1(it):
        norm2.set(norm2 + A[it, n-1] ** 2)
    tf.loop(loop_body1, 0, m, 1)
    R[n-1, n-1] = tf.sqrt(norm2)
    Q[j, n-1] = A[j, n-1] / R[n-1, n-1]

    return [Q, R]

qr = tf.compile(QRDecomposition)


QRDecomposition:
  Kernel count: 8
  Intermediate buffers: 0
  Lines of generated code: 578



In [19]:
#generate random matrix
A = np.random.rand(5, 5)

#compute QR decomposition
Q, R = modified_gram_schmidt(A)

#compute QR decomposition using TensorFrost
Atf = tf.tensor(A)
Qtf, Rtf = qr(Atf)
Qnp = Qtf.numpy
Rnp = Rtf.numpy

#check if QR decomposition is correct
print("QR decomposition is correct:", np.allclose(A, np.dot(Q, R)))
print("QR decomposition using TensorFrost is correct:", np.allclose(A, np.dot(Qnp, Rnp)))

#check error
print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qnp, Rnp)))

#print Q and R
print("Q:\n", Qnp)
print("R:\n", Rnp)


QR decomposition is correct: True
QR decomposition using TensorFrost is correct: True
Error: 2.607399605677454e-16
Error using TensorFrost: 1.1614004081603156e-07
Q:
 [[ 6.3341695e-01 -1.8382910e-01  3.3214268e-01 -1.4583039e-04
  -6.7429304e-01]
 [ 4.1568449e-01  4.7665125e-01 -6.1356419e-01 -4.7097704e-01
  -4.1588847e-02]
 [ 5.8963472e-01 -4.2631903e-01 -2.2058693e-01  3.2675329e-01
   5.6138843e-01]
 [ 2.5641686e-01  3.8103092e-01  6.8158746e-01 -3.1775081e-01
   4.7279873e-01]
 [ 1.1211982e-01  6.4193302e-01 -3.6244455e-04  7.5527966e-01
  -7.0025690e-02]]
R:
 [[ 1.5585067   1.3178939   1.1389446   0.80471003  1.2012088 ]
 [ 0.          0.7865809   0.5951857   0.80946875  0.4737573 ]
 [ 0.          0.          0.37996578 -0.09868313 -0.14027269]
 [ 0.          0.          0.          0.45774737 -0.41879633]
 [ 0.          0.          0.          0.          0.33441728]]


In [20]:
#performance test
import time
A = np.random.rand(750, 750).astype(np.float32)

#naive NumPy QR decomposition
start = time.time()
Q, R = modified_gram_schmidt(A)
print("Time for naive NumPy QR decomposition:", time.time() - start)

#TensorFrost QR decomposition
Atf = tf.tensor(A)
start = time.time()
Qtf, Rtf = qr(Atf)
print("Time for TensorFrost QR decomposition:", time.time() - start)

#built-in NumPy QR decomposition
start = time.time()
Q, R = np.linalg.qr(A)
print("Time for built-in NumPy QR decomposition:", time.time() - start)

#householder QR decomposition
start = time.time()
Q, R = qr_decomposition(A)
print("Time for householder QR decomposition:", time.time() - start)

print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qtf.numpy, Rtf.numpy)))

Time for naive NumPy QR decomposition: 1.0544276237487793
Time for TensorFrost QR decomposition: 0.16899871826171875
Time for built-in NumPy QR decomposition: 0.10702252388000488
Time for householder QR decomposition: 10.901634693145752
Error: 0.0001499538873179337
Error using TensorFrost: 0.00016575323
